Go from an image directory of images to a dataset directory structure.

In [1]:
# Import stuff
import numpy as np
import os
import shutil
from datetime import timedelta
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [2]:
# Directory address
# image_dir = "/blue/azarrinpar/ICG_AI/pytorch-CycleGAN-and-pix2pix/img_src/9-23/"
image_dir = "/blue/azarrinpar/ICG_AI/ModelData/2024_11_24_Older_BothDose/"

# Dataset address
dataset_dir = "/blue/azarrinpar/ICG_AI/pytorch-CycleGAN-and-pix2pix/datasets/2024_11_24_Older_BothDose/"

In [3]:
# Get list of images names in the directory and check they are the same
vis_dir = image_dir + "vis/"
vis_list = sorted(os.listdir(vis_dir))
icg_dir = image_dir + "icg/"
icg_list = sorted(os.listdir(icg_dir))
assert vis_list == icg_list, "The two directories do not have the same images"
img_list = vis_list

In [4]:
print(img_list)

['P10S1.png', 'P10S2.png', 'P10S3.png', 'P10S4.png', 'P10S5.png', 'P11S1.png', 'P11S2.png', 'P11S3.png', 'P11S4.png', 'P11S5.png', 'P11S6.png', 'P12S1.png', 'P12S2.png', 'P12S3.png', 'P12S4.png', 'P12S5.png', 'P12S6.png', 'P13S1.png', 'P13S2.png', 'P13S3.png', 'P13S4.png', 'P13S5.png', 'P15S1.png', 'P15S2.png', 'P15S3.png', 'P15S4.png', 'P15S5.png', 'P16S1.png', 'P16S2.png', 'P16S3.png', 'P16S4.png', 'P16S5.png', 'P16S6.png', 'P17S1.png', 'P17S2.png', 'P17S3.png', 'P17S4.png', 'P17S5.png', 'P18S1.png', 'P18S2.png', 'P18S3.png', 'P18S4.png', 'P18S5.png', 'P18S6.png', 'P19S1.png', 'P19S2.png', 'P19S3.png', 'P19S4.png', 'P19S5.png', 'P1S1.png', 'P1S2.png', 'P1S3.png', 'P1S4.png', 'P1S5.png', 'P1S6.png', 'P21S1.png', 'P21S2.png', 'P21S3.png', 'P21S4.png', 'P21S5.png', 'P22S1.png', 'P22S2.png', 'P22S3.png', 'P22S4.png', 'P22S5.png', 'P22S6.png', 'P23S1.png', 'P23S2.png', 'P23S3.png', 'P23S4.png', 'P23S5.png', 'P24S1.png', 'P24S2.png', 'P24S3.png', 'P24S4.png', 'P24S5.png', 'P24S6.png', 'P25

In [5]:
# Make directories for the dataset
vis_train = os.path.join(dataset_dir, "A/train")
vis_test = os.path.join(dataset_dir, "A/test")
vis_val = os.path.join(dataset_dir, "A/val")
icg_train = os.path.join(dataset_dir, "B/train")
icg_test = os.path.join(dataset_dir, "B/test")
icg_val = os.path.join(dataset_dir, "B/val")

# Create directories
os.makedirs(vis_train, exist_ok=True)
os.makedirs(vis_test, exist_ok=True)
os.makedirs(vis_val, exist_ok=True)
os.makedirs(icg_train, exist_ok=True)
os.makedirs(icg_test, exist_ok=True)
os.makedirs(icg_val, exist_ok=True)

In [6]:
# Randomize the image list if you want
np.random.seed(0)
# np.random.shuffle(img_list)
print(img_list)

['P10S1.png', 'P10S2.png', 'P10S3.png', 'P10S4.png', 'P10S5.png', 'P11S1.png', 'P11S2.png', 'P11S3.png', 'P11S4.png', 'P11S5.png', 'P11S6.png', 'P12S1.png', 'P12S2.png', 'P12S3.png', 'P12S4.png', 'P12S5.png', 'P12S6.png', 'P13S1.png', 'P13S2.png', 'P13S3.png', 'P13S4.png', 'P13S5.png', 'P15S1.png', 'P15S2.png', 'P15S3.png', 'P15S4.png', 'P15S5.png', 'P16S1.png', 'P16S2.png', 'P16S3.png', 'P16S4.png', 'P16S5.png', 'P16S6.png', 'P17S1.png', 'P17S2.png', 'P17S3.png', 'P17S4.png', 'P17S5.png', 'P18S1.png', 'P18S2.png', 'P18S3.png', 'P18S4.png', 'P18S5.png', 'P18S6.png', 'P19S1.png', 'P19S2.png', 'P19S3.png', 'P19S4.png', 'P19S5.png', 'P1S1.png', 'P1S2.png', 'P1S3.png', 'P1S4.png', 'P1S5.png', 'P1S6.png', 'P21S1.png', 'P21S2.png', 'P21S3.png', 'P21S4.png', 'P21S5.png', 'P22S1.png', 'P22S2.png', 'P22S3.png', 'P22S4.png', 'P22S5.png', 'P22S6.png', 'P23S1.png', 'P23S2.png', 'P23S3.png', 'P23S4.png', 'P23S5.png', 'P24S1.png', 'P24S2.png', 'P24S3.png', 'P24S4.png', 'P24S5.png', 'P24S6.png', 'P25

In [7]:
TEST_PERCENT = 0.1
VAL_PERCENT = 0.2

# Split the image list into 
# train, test, and validation sets using sklearn ttv split
train_list, test_list = train_test_split(img_list, test_size=TEST_PERCENT, shuffle=False)
train_list, val_list = train_test_split(train_list, test_size=VAL_PERCENT, shuffle=False)
print("Train list: ", train_list)
print("Val list: ", val_list)
print("Test list: ", test_list)

Train list:  ['P10S1.png', 'P10S2.png', 'P10S3.png', 'P10S4.png', 'P10S5.png', 'P11S1.png', 'P11S2.png', 'P11S3.png', 'P11S4.png', 'P11S5.png', 'P11S6.png', 'P12S1.png', 'P12S2.png', 'P12S3.png', 'P12S4.png', 'P12S5.png', 'P12S6.png', 'P13S1.png', 'P13S2.png', 'P13S3.png', 'P13S4.png', 'P13S5.png', 'P15S1.png', 'P15S2.png', 'P15S3.png', 'P15S4.png', 'P15S5.png', 'P16S1.png', 'P16S2.png', 'P16S3.png', 'P16S4.png', 'P16S5.png', 'P16S6.png', 'P17S1.png', 'P17S2.png', 'P17S3.png', 'P17S4.png', 'P17S5.png', 'P18S1.png', 'P18S2.png', 'P18S3.png', 'P18S4.png', 'P18S5.png', 'P18S6.png', 'P19S1.png', 'P19S2.png', 'P19S3.png', 'P19S4.png', 'P19S5.png', 'P1S1.png', 'P1S2.png', 'P1S3.png', 'P1S4.png', 'P1S5.png', 'P1S6.png', 'P21S1.png', 'P21S2.png', 'P21S3.png', 'P21S4.png', 'P21S5.png', 'P22S1.png', 'P22S2.png', 'P22S3.png', 'P22S4.png', 'P22S5.png', 'P22S6.png', 'P23S1.png', 'P23S2.png', 'P23S3.png', 'P23S4.png', 'P23S5.png', 'P24S1.png', 'P24S2.png', 'P24S3.png', 'P24S4.png', 'P24S5.png', 'P24

In [8]:
# Copy the images to the directories
for img in tqdm(train_list, desc="Copying train images"):
    shutil.copy(os.path.join(vis_dir, img), vis_train)
    shutil.copy(os.path.join(icg_dir, img), icg_train)
    # vis_img = cv2.imread(os.path.join(vis_dir, img))
    # icg_img = cv2.imread(os.path.join(icg_dir, img))
    # cv2.imwrite(os.path.join(vis_train, img), vis_img)
    # cv2.imwrite(os.path.join(icg_train, img), icg_img)

for img in tqdm(val_list, desc="Copying val images"):
    shutil.copy(os.path.join(vis_dir, img), vis_val)
    shutil.copy(os.path.join(icg_dir, img), icg_val)
    # vis_img = cv2.imread(os.path.join(vis_dir, img))
    # icg_img = cv2.imread(os.path.join(icg_dir, img))
    # cv2.imwrite(os.path.join(vis_val, img), vis_img)
    # cv2.imwrite(os.path.join(icg_val, img), icg_img)

for img in tqdm(test_list, desc="Copying test images"):
    shutil.copy(os.path.join(vis_dir, img), vis_test)
    shutil.copy(os.path.join(icg_dir, img), icg_test)
    # vis_img = cv2.imread(os.path.join(vis_dir, img))
    # icg_img = cv2.imread(os.path.join(icg_dir, img))
    # cv2.imwrite(os.path.join(vis_test, img), vis_img)
    # cv2.imwrite(os.path.join(icg_test, img), icg_img)

Copying test images: 100%|██████████| 15/15 [00:00<00:00, 181.41it/s]


In [10]:
# FOR img2img-turbo
# Create a json for train, test, and val "prompts"
# Each json is a dictionary where the filename is the key and the prompt string is the value
# Let us make this with all of the prompts being empty strings

# train.json
train_json = {}
for img in train_list:
    train_json[img] = ""
# Let us format it with every dictionary entry on a new line; this did not succeed yet
train_json = pd.Series(train_json)
train_json.to_json(dataset_dir + "train.json")

# test.json
test_json = {}
for img in test_list:
    test_json[img] = ""
test_json = pd.Series(test_json)
test_json.to_json(dataset_dir + "test.json")